In [ ]:
import os
import shutil
import json
import datetime

base_dir = "full_sweep_v2"
new_dir = f"{base_dir}_organized"
date = "1230"

# We’ll loop through each folder under `base_dir`
# that starts with "resid_post_layer_*".
for folder in os.listdir(base_dir):
    if folder.startswith("resid_post_layer_"):
        full_subdir_path = os.path.join(base_dir, folder)
        
        # Skip if it's not actually a directory
        if not os.path.isdir(full_subdir_path):
            continue
        
        # For each trainer directory (e.g. trainer_0, trainer_1...)
        for trainer_dir in os.listdir(full_subdir_path):
            trainer_path = os.path.join(full_subdir_path, trainer_dir)
            
            # Skip if it's not a folder
            if not os.path.isdir(trainer_path):
                continue
            
            # We expect a config.json inside each trainer directory
            config_path = os.path.join(trainer_path, "config.json")
            if not os.path.isfile(config_path):
                continue  # If no config.json, skip
            
            # Read the config file
            with open(config_path, "r") as f:
                config = json.load(f)
            
            # Extract the trainer class
            trainer_class = config["trainer"]["trainer_class"]
            model_name = config["trainer"]["lm_name"]
            ctx_len = config["buffer"]["ctx_len"]

            new_folder = f"{trainer_class}_{model_name}_ctx{ctx_len}_{date}".replace("/", "_")
            
            # Construct the target path: <TrainerClass>/<resid_post_layer_X>/<trainer_#>
            new_folder_path = os.path.join(new_dir, new_folder, folder)
            new_trainer_path = os.path.join(new_folder_path, trainer_dir)
            
            # Make sure the new folder exists
            os.makedirs(new_folder_path, exist_ok=True)
            
            # Copy (instead of move) the entire trainer directory
            shutil.copytree(trainer_path, new_trainer_path, dirs_exist_ok=True)
            
            print(f"Copied {trainer_path} -> {new_trainer_path}")

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="full_sweep_v2_organized",
    path_in_repo="",
    repo_id="adamkarvonen/sae_test",
    repo_type="model",
    # allow_patterns="*eval_results.json"
)